In [ ]:
import pandas as pd
data_final = pd.read_csv('finaldata_combined.csv')
data_final.head(5)

,Date,Price,Open,High,Low,Vol.,Change %,Symbol,Nltk_Sentiment,FinBERT_Sentiment,Subjectivity_avg,Polarity_avg,PriceSP500,OpenSP500,HighSP500,LowSP500,Change%SP500
0,2024-04-26,21550.0,21300.0,21550.0,21070.0,167.50K,2.38%,E1VFVN30,0.3352,0.0006,0.4333,0.1375,129748.08236,129358.58065,130121.04742,129065.75474,1.02%
1,2024-04-25,21050.0,21300.0,21300.0,20100.0,792.60K,-1.03%,E1VFVN30,-0.0258,-0.9995,0.4517,0.1067,128436.85322,127710.76708,128674.21775,126965.34578,-0.46%
2,2024-04-24,21270.0,20750.0,21300.0,20750.0,125.00K,2.51%,E1VFVN30,0.0000,0.9999,0.3333,0.2500,129027.33883,129363.92326,129481.46068,128401.23582,0.02%
3,2024-04-23,20750.0,20940.0,20940.0,20520.0,701.80K,-0.86%,E1VFVN30,0.6808,0.9990,0.2000,-0.0500,128999.86255,127938.97285,129141.56892,127916.33036,1.20%
4,2024-04-22,20930.0,20670.0,20930.0,20670.0,638.20K,1.26%,E1VFVN30,0.6124,1.0000,0.4292,0.1958,127474.67460,126882.66253,128193.12844,126426.50540,0.87%


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dropout, Dense, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from math import sqrt
from joblib import dump
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.dates as mdates

# Load data
data = pd.read_csv('finaldata_combined.csv')

# Function to preprocess data
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to preprocess data
def process_data(data):
    # Chuyển đổi 'Vol.' từ chuỗi sang số thực
    data['Vol.'] = data['Vol.'].str.replace('K', 'e3').str.replace('M', 'e6').str.replace('B', 'e9').astype(float)

    # Chuyển đổi 'Change %' và 'Change%SP500' thành số thực và loại bỏ ký tự '%'
    data['Change %'] = data['Change %'].str.rstrip('%').astype(float) / 100
    data['Change%SP500'] = data['Change%SP500'].str.rstrip('%').astype(float) / 100

    # Nếu cột 'FinBERT_Sentiment' tồn tại, chuyển đổi dữ liệu thành số nguyên
    if 'FinBERT_Sentiment' in data.columns:
        data['FinBERT_Sentiment'] = pd.to_numeric(data['FinBERT_Sentiment'], errors='coerce', downcast='integer')

    # Chuyển đổi cột 'Date' sang dạng thời gian
    data['Date'] = pd.to_datetime(data['Date'])

    # Nhóm dữ liệu theo 'Symbol', sau đó nhóm theo tuần và tính giá trị trung bình
    data[['Open', 'High', 'Low', 'Vol.', 'Change %', 'PriceSP500', 'OpenSP500', 'HighSP500', 'LowSP500', 'Change%SP500']] = \
        data.groupby(['Symbol', data['Date'].dt.isocalendar().week]) \
            [['Open', 'High', 'Low', 'Vol.', 'Change %', 'PriceSP500', 'OpenSP500', 'HighSP500', 'LowSP500', 'Change%SP500']] \
            .transform(lambda x: x.fillna(x.mean()))

    return data

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dropout, Dense, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from math import sqrt
from joblib import dump
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.dates as mdates

# Load data
data = pd.read_csv('finaldata_combined.csv')

# Function to preprocess data
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to preprocess data
def process_data(data):
    # Chuyển đổi 'Vol.' từ chuỗi sang số thực
    data['Vol.'] = data['Vol.'].str.replace('K', 'e3').str.replace('M', 'e6').str.replace('B', 'e9').astype(float)

    # Chuyển đổi 'Change %' và 'Change%SP500' thành số thực và loại bỏ ký tự '%'
    data['Change %'] = data['Change %'].str.rstrip('%').astype(float) / 100
    data['Change%SP500'] = data['Change%SP500'].str.rstrip('%').astype(float) / 100

    # Nếu cột 'FinBERT_Sentiment' tồn tại, chuyển đổi dữ liệu thành số nguyên
    if 'FinBERT_Sentiment' in data.columns:
        data['FinBERT_Sentiment'] = pd.to_numeric(data['FinBERT_Sentiment'], errors='coerce', downcast='integer')

    # Chuyển đổi cột 'Date' sang dạng thời gian
    data['Date'] = pd.to_datetime(data['Date'])

    # Nhóm dữ liệu theo 'Symbol', sau đó nhóm theo tuần và tính giá trị trung bình
    data[['Open', 'High', 'Low', 'Vol.', 'Change %', 'PriceSP500', 'OpenSP500', 'HighSP500', 'LowSP500', 'Change%SP500']] = \
        data.groupby(['Symbol', data['Date'].dt.isocalendar().week]) \
            [['Open', 'High', 'Low', 'Vol.', 'Change %', 'PriceSP500', 'OpenSP500', 'HighSP500', 'LowSP500', 'Change%SP500']] \
            .transform(lambda x: x.fillna(x.mean()))

    return data
# Function to create sequences for LSTM input
def create_sequences(features, target, seq_length):
    xs, ys = [], []
    for i in range(len(features) - seq_length):
        x = features[i:(i + seq_length)]
        y = target[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define LSTM model
def LSTM_model(input_shape):
    model = Sequential([
        LSTM(units=100, return_sequences=True, input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.2),
        LSTM(units=50, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(units=50),
        BatchNormalization(),
        Dropout(0.5),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Define Bi-LSTM model
def BiLSTM_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(units=100, return_sequences=True), input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.2),
        Bidirectional(LSTM(units=50, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.2),
        Bidirectional(LSTM(units=50)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
# Tạo một DataFrame để lưu trữ thông tin về các model tốt nhất và tệ nhất
best_worst_models = pd.DataFrame(columns=['Symbol', 'Case', 'Model_Type', 'Metric', 'Value'])

# Hàm cập nhật bảng thông tin về model tốt nhất và tệ nhất
def update_best_worst_models(symbol, case, model_type, metric, value):
    global best_worst_models

    if best_worst_models.empty or not ((best_worst_models['Symbol'] == symbol) & (best_worst_models['Case'] == case) & (best_worst_models['Model_Type'] == model_type) & (best_worst_models['Metric'] == metric)).any():
        best_worst_models.loc[len(best_worst_models)] = {'Symbol': symbol, 'Case': case, 'Model_Type': model_type, 'Metric': metric, 'Value': value}
    else:
        row_index = best_worst_models[(best_worst_models['Symbol'] == symbol) & (best_worst_models['Case'] == case) & (best_worst_models['Model_Type'] == model_type) & (best_worst_models['Metric'] == metric)].index[0]

        if metric in ['R2']:
            if value < best_worst_models.at[row_index, 'Value']:
                best_worst_models.at[row_index, 'Value'] = value
        elif metric in ['R2']:
            if value > best_worst_models.at[row_index, 'Value']:
                best_worst_models.at[row_index, 'Value'] = value
def plot_evaluation_metrics(lstm_model, bi_lstm_model, lstm_history, bi_lstm_history, X_test, y_test, scaler_target, symbol, test_data, seq_length):
    # Predict Price LSTM
    y_pred_lstm = lstm_model.predict(X_test)
    y_pred_lstm_original = scaler_target.inverse_transform(y_pred_lstm)
    y_test_original = scaler_target.inverse_transform(y_test)

    # Predict Price with Bi-LSTM
    y_pred_bilstm = bi_lstm_model.predict(X_test)
    y_pred_bilstm_original = scaler_target.inverse_transform(y_pred_bilstm)

    # Plot Loss
    plt.figure(figsize=(18, 10))

    plt.subplot(2, 2, 1)
    plt.plot(lstm_history.history['loss'], label='LSTM train')
    plt.plot(lstm_history.history['val_loss'], label='LSTM validation')
    plt.plot(bi_lstm_history.history['loss'], label='Bi-LSTM train')
    plt.plot(bi_lstm_history.history['val_loss'], label='Bi-LSTM validation')
    plt.title(f'Loss - {symbol}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Đánh giá mô hình và vẽ sơ đồ cột cho MSE, RMSE, MAE
    plt.subplot(2, 2, 2)
    metrics = ['MSE', 'RMSE', 'MAE']
    lstm_metrics = evaluate_model(lstm_model, X_test, y_test, scaler_target, symbol, case, 'LSTM')[:3]
    bi_lstm_metrics = evaluate_model(bi_lstm_model, X_test, y_test, scaler_target, symbol, case, 'Bi-LSTM')[:3]

    index = np.arange(len(metrics))
    bar_width = 0.35

    lstm_bars = plt.bar(index, lstm_metrics, bar_width, label='LSTM', color='blue')
    bi_lstm_bars = plt.bar(index + bar_width, bi_lstm_metrics, bar_width, label='Bi-LSTM', color='orange')

    plt.xlabel('Metrics')
    plt.ylabel('Values')
    plt.title(f'Model Evaluation Metrics - {symbol}')
    plt.xticks(index + bar_width / 2, metrics)
    plt.legend()

    # Plot Correlation Coefficient
    plt.subplot(2, 2, 3)
    lstm_corr_coef = np.corrcoef(y_test.flatten(), y_pred_lstm.flatten())[0, 1]
    bi_lstm_corr_coef = np.corrcoef(y_test.flatten(), y_pred_bilstm.flatten())[0, 1]
    plt.scatter(y_test, y_pred_lstm, label=f'LSTM (corr={lstm_corr_coef:.2f})', alpha=0.5)
    plt.scatter(y_test, y_pred_bilstm, label=f'Bi-LSTM (corr={bi_lstm_corr_coef:.2f})', alpha=0.5)
    plt.title(f'Correlation Coefficient - {symbol}')
    plt.xlabel('True Price')
    plt.ylabel('Predicted Price')
    plt.legend()

    # Plot Stock Price with Date on X-axis
    plt.subplot(2, 2, 4)
    test_dates = test_data['Date'].iloc[seq_length:]  # Adjusted test dates
    test_actual_prices = test_data['Price'].iloc[seq_length:]  # Adjusted test prices
    plt.plot(test_dates, test_actual_prices, label='Actual Price', color='black', linestyle='-')
    plt.plot(test_dates, y_pred_lstm_original, label='LSTM Predicted Price', color='blue', linestyle='-')
    plt.plot(test_dates, y_pred_bilstm_original, label='Bi-LSTM Predicted Price', color='orange', linestyle='-')
    plt.title(f'Predicted vs Actual Stock Prices - {symbol}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()






# Hàm đánh giá mô hình và vẽ sơ đồ cột
def evaluate_model(model, X_test, y_test, scaler_target, symbol, case, model_type):
    metrics = ['MSE', 'RMSE', 'MAE', 'R2']

    y_pred = model.predict(X_test)
    y_pred_original = scaler_target.inverse_transform(y_pred)
    y_test_original = scaler_target.inverse_transform(y_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)  # RMSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test_original, y_pred_original)

    # Cập nhật thông tin về model tốt nhất và tệ nhất
    update_best_worst_models(symbol, case, model_type, 'MSE', mse)
    update_best_worst_models(symbol, case, model_type, 'RMSE', rmse)
    update_best_worst_models(symbol, case, model_type, 'MAE', mae)
    update_best_worst_models(symbol, case, model_type, 'R2', r2)

    return mse, rmse, mae, r2
# Process data
data = process_data(data)

# Function to process and train model
def process_and_train_model(data, symbol, case=1, seq_length=5, num_epochs=2):
    # Filter data for training and validation (excluding 2024)
    train_data = data[data['Date'].dt.year != 2024]
    # Filter data for testing (only 2024)
    test_data = data[data['Date'].dt.year == 2024]

    data_symbol = train_data[train_data['Symbol'] == symbol].copy()
    data_2024_symbol = test_data[test_data['Symbol'] == symbol].copy()

    if case == 1:
        features_columns = ['High', 'Low', 'Open', 'Price','Change %', 'Vol.']
    elif case == 2:
        features_columns = ['Open', 'High', 'Low', 'Change %', 'Price', 'Vol.', 'HighSP500', 'LowSP500', 'OpenSP500', 'PriceSP500', 'Change%SP500']
    elif case == 3:
        features_columns = ['Subjectivity_avg','Polarity_avg', 'FinBERT_Sentiment', 'Nltk_Sentiment','High', 'Low', 'Open', 'Price','Change %', 'Vol.' ]
    elif case == 4:
        features_columns = ['Subjectivity_avg','Polarity_avg', 'FinBERT_Sentiment', 'Nltk_Sentiment', 'Price', 'Open', 'High', 'Low', 'Change %', 'Vol.', 'HighSP500', 'LowSP500', 'OpenSP500', 'PriceSP500', 'Change%SP500']

    target_column = 'Price'

    features = data_symbol[features_columns].values
    target = data_symbol[target_column].values.reshape(-1, 1)

    scalers = {
        'features': MinMaxScaler(feature_range=(0,1)),
        'target': MinMaxScaler(feature_range=(0,1))
    }
    features_scaled = scalers['features'].fit_transform(features)
    target_scaled = scalers['target'].fit_transform(target)

    X_train, y_train = create_sequences(features_scaled, target_scaled, seq_length)
    input_shape = (X_train.shape[1], X_train.shape[2])

    lstm_model = LSTM_model(input_shape)
    lstm_history = lstm_model.fit(X_train, y_train, epochs=num_epochs, batch_size=256, validation_split=0.1)
    lstm_model_path = f'{symbol}_lstm_model_{case}.h5'
    lstm_model.save(lstm_model_path)
    dump(scalers, f'{symbol}_lstm_scalers_{case}.joblib')
    dump(scalers['target'], f'{symbol}_lstm_target_scaler_{case}.joblib')
    bi_lstm_model = BiLSTM_model(input_shape)
    bi_lstm_history = bi_lstm_model.fit(X_train, y_train, epochs=num_epochs, batch_size=256, validation_split=0.1)
    bilstm_model_path = f'{symbol}_bilstm_model_{case}.h5'
    bi_lstm_model.save(bilstm_model_path)
    dump(scalers, f'{symbol}_bilstm_scalers_{case}.joblib')
    dump(scalers['target'], f'{symbol}_bilstm_target_scaler_{case}.joblib')

    features_2024 = data_2024_symbol[features_columns].values
    features_2024_scaled = scalers['features'].transform(features_2024)
    target_2024 = data_2024_symbol[target_column].values.reshape(-1, 1)
    target_2024_scaled = scalers['target'].transform(target_2024)
    # Check if there are enough rows to create sequences
    if len(features_2024) <= seq_length:
        adjusted_seq_length = len(features_2024) - 1
    else:
        adjusted_seq_length = seq_length

    X_test, y_test = create_sequences(features_2024_scaled, target_2024_scaled, adjusted_seq_length)

    plot_evaluation_metrics(lstm_model, bi_lstm_model, lstm_history, bi_lstm_history, X_test, y_test, scalers['target'], symbol, data_2024_symbol, adjusted_seq_length)

    return lstm_model, lstm_history, bi_lstm_model, bi_lstm_history, X_test, y_test, scalers['target'], seq_length

# best_models = {}
# worst_models = {}

# symbols = data['Symbol'].unique()
# cases = [1, 2, 3, 4]
# for symbol in symbols:
#     print(f"Processing {symbol}")
#     for case in cases:
#         print(f"Case {case}")
#         lstm_model, lstm_history, bi_lstm_model, bi_lstm_history, X_test, y_test, scaler_target, seq_length = process_and_train_model(data, symbol, case=case)
#         lstm_mse, lstm_rmse, lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test, scaler_target, symbol, case, 'LSTM')
#         bi_lstm_mse, bi_lstm_rmse, bi_lstm_mae, bi_lstm_r2 = evaluate_model(bi_lstm_model, X_test, y_test, scaler_target, symbol, case, 'Bi-LSTM')
#         print(f"In case {case} :{symbol} LSTM have MSE={lstm_mse}, MAE={lstm_mae}, RMSE={lstm_rmse}")
#         print(f"In case {case} :{symbol} BiLSTM have MSE={bi_lstm_mse}, MAE={bi_lstm_mae}, RMSE={bi_lstm_rmse}")
#         if case not in best_models or lstm_r2 > best_models[case]['best'][5]:
#             best_models[case] = {'best': ('LSTM', symbol, lstm_mse, lstm_rmse, lstm_mae, lstm_r2)}
#         if case not in worst_models or bi_lstm_r2 < worst_models[case]['worst'][5]:
#             worst_models[case] = {'worst': ('Bi-LSTM', symbol, bi_lstm_mse, bi_lstm_rmse, bi_lstm_mae, bi_lstm_r2)}

####

# for case, model_info in best_models.items():
#     print(f"In case {case}:")
#     print(f"Best model is {model_info['best'][0]}, symbol is {model_info['best'][1]}, MSE is {model_info['best'][2]}, RMSE is {model_info['best'][3]}, MAE is {model_info['best'][4]}, R2 is {model_info['best'][5]}.")

# for case, model_info in worst_models.items():
#     print(f"In case {case}:")
#     print(f"Worst model is {model_info['worst'][0]}, symbol is {model_info['worst'][1]}, MSE is {model_info['worst'][2]}, RMSE is {model_info['worst'][3]}, MAE is {model_info['worst'][4]}, R2 is {model_info['worst'][5]}.")
# for case, model_info in best_models.items():
#     print(f"In case {case}:")
#     print(f"Best model is {model_info['best'][0]}, symbol is {model_info['best'][1]}, MSE is {model_info['best'][2]}, RMSE is {model_info['best'][3]}, MAE is {model_info['best'][4]}.")

# for case, model_info in worst_models.items():
#     print(f"In case {case}:")
#     print(f"Worst model is {model_info['worst'][0]}, symbol is {model_info['worst'][1]}, MSE is {model_info['worst'][2]}, RMSE is {model_info['worst'][3]}, MAE is {model_info['worst'][4]}.")


######
symbols = data['Symbol'].unique()
cases = [1, 2, 3, 4]
model_info_list = {case: [] for case in cases}

for symbol in symbols:
    print(f"Processing {symbol}")
    for case in cases:
        print(f"Case {case}")
        lstm_model, lstm_history, bi_lstm_model, bi_lstm_history, X_test, y_test, scaler_target, seq_length = process_and_train_model(data, symbol, case=case)

        # Evaluate LSTM model
        lstm_mse, lstm_rmse, lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test, scaler_target, symbol, case, 'LSTM')
        # Evaluate Bi-LSTM model
        bi_lstm_mse, bi_lstm_rmse, bi_lstm_mae, bi_lstm_r2 = evaluate_model(bi_lstm_model, X_test, y_test, scaler_target, symbol, case, 'Bi-LSTM')

        print(f"In case {case}: {symbol} LSTM have MSE={lstm_mse}, MAE={lstm_mae}, RMSE={lstm_rmse}")
        print(f"In case {case}: {symbol} BiLSTM have MSE={bi_lstm_mse}, MAE={bi_lstm_mae}, RMSE={bi_lstm_rmse}")

        # Store model information
        model_info_list[case].append({
            'model_type': 'LSTM',
            'symbol': symbol,
            'mse': lstm_mse,
            'rmse': lstm_rmse,
            'mae': lstm_mae,
            # 'r2': lstm_r2
        })
        model_info_list[case].append({
            'model_type': 'Bi-LSTM',
            'symbol': symbol,
            'mse': bi_lstm_mse,
            'rmse': bi_lstm_rmse,
            'mae': bi_lstm_mae,
            # 'r2': bi_lstm_r2
        })

# Print all model information for each case
for case, models in model_info_list.items():
    print(f"\nIn case {case}:")
    for model_info in models:
        print(f"{model_info['model_type']} model for symbol {model_info['symbol']} - MSE: {model_info['mse']}, RMSE: {model_info['rmse']}, MAE: {model_info['mae']}")


Processing E1VFVN30
Case 1
Epoch 1/2


KeyboardInterrupt: 

In [6]:
from keras.layers import Input, LSTM, Dropout, Dense, BatchNormalization, Attention
from keras.models import Model
import pandas as pd

def LSTM_with_SelfAttention_model(input_shape):
    inputs = Input(shape=input_shape)

    x = LSTM(units=100, return_sequences=True)(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = LSTM(units=50, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    attention_output = Attention()([x, x])  # Self-attention
    x = LSTM(units=50)(attention_output)

    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(units=1)(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model
def BiLSTM_with_SelfAttention_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # Bi-directional LSTM layers with return_sequences=True for attention
    x = Bidirectional(LSTM(units=100, return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(units=50, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Applying Attention mechanism
    attention_output = Attention()([x, x])  # Self-attention
    x = Bidirectional(LSTM(units=50))(attention_output)

    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Output layer
    outputs = Dense(units=1)(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model




In [ ]:
def process_and_train_model(data, symbol, case=1, seq_length=5, num_epochs=2):
    # Filter data for training and validation (excluding 2024)
    train_data = data[data['Date'].dt.year != 2024]
    # Filter data for testing (only 2024)
    test_data = data[data['Date'].dt.year == 2024]

    data_symbol = train_data[train_data['Symbol'] == symbol].copy()
    data_2024_symbol = test_data[test_data['Symbol'] == symbol].copy()

    if case == 1:
        features_columns = ['High', 'Low', 'Open', 'Price','Change %', 'Vol.']
    elif case == 2:
        features_columns = ['Open', 'High', 'Low', 'Change %', 'Price', 'Vol.', 'HighSP500', 'LowSP500', 'OpenSP500', 'PriceSP500', 'Change%SP500']
    elif case == 3:
        features_columns = ['Subjectivity_avg','Polarity_avg', 'FinBERT_Sentiment', 'Nltk_Sentiment','High', 'Low', 'Open', 'Price','Change %', 'Vol.' ]
    elif case == 4:
        features_columns = ['Subjectivity_avg','Polarity_avg', 'FinBERT_Sentiment', 'Nltk_Sentiment', 'Price', 'Open', 'High', 'Low', 'Change %', 'Vol.', 'HighSP500', 'LowSP500', 'OpenSP500', 'PriceSP500', 'Change%SP500']

    target_column = 'Price'

    features = data_symbol[features_columns].values
    target = data_symbol[target_column].values.reshape(-1, 1)

    scalers = {
        'features': MinMaxScaler(feature_range=(0,1)),
        'target': MinMaxScaler(feature_range=(0,1))
    }
    features_scaled = scalers['features'].fit_transform(features)
    target_scaled = scalers['target'].fit_transform(target)

    X_train, y_train = create_sequences(features_scaled, target_scaled, seq_length)
    input_shape = (X_train.shape[1], X_train.shape[2])

    self_attention_lstm_model = LSTM_with_SelfAttention_model(input_shape)
    self_attention_lstm_history = self_attention_lstm_model.fit(X_train, y_train, epochs=num_epochs, batch_size=256, validation_split=0.1)
    self_attention_lstm_model_path = f'{symbol}_self_attention_lstm_model_{case}.h5'
    self_attention_lstm_model.save(self_attention_lstm_model_path)
    dump(scalers, f'{symbol}_self_attention_lstm_scalers_{case}.joblib')
    dump(scalers['target'], f'{symbol}_self_attention_lstm_target_scaler_{case}.joblib')

    self_attention_bi_lstm_model = BiLSTM_with_SelfAttention_model(input_shape)
    self_attention_bi_lstm_history = self_attention_bi_lstm_model.fit(X_train, y_train, epochs=num_epochs, batch_size=256, validation_split=0.1)
    self_attention_bilstm_model_path = f'{symbol}_self_attention_bilstm_model_{case}.h5'
    self_attention_bi_lstm_model.save(self_attention_bilstm_model_path)
    dump(scalers, f'{symbol}_self_attention_bilstm_scalers_{case}.joblib')
    dump(scalers['target'], f'{symbol}_self_attention_bilstm_target_scaler_{case}.joblib')

    features_2024 = data_2024_symbol[features_columns].values
    features_2024_scaled = scalers['features'].transform(features_2024)
    target_2024 = data_2024_symbol[target_column].values.reshape(-1, 1)
    target_2024_scaled = scalers['target'].transform(target_2024)
    # Check if there are enough rows to create sequences
    if len(features_2024) <= seq_length:
        adjusted_seq_length = len(features_2024) - 1
    else:
        adjusted_seq_length = seq_length

    X_test, y_test = create_sequences(features_2024_scaled, target_2024_scaled, adjusted_seq_length)

    plot_evaluation_metrics(self_attention_lstm_model, self_attention_bi_lstm_model, self_attention_lstm_history, self_attention_bi_lstm_history, X_test, y_test, scalers['target'], symbol, data_2024_symbol, adjusted_seq_length)

    return self_attention_lstm_model, self_attention_lstm_history, self_attention_bi_lstm_model, self_attention_bi_lstm_history, X_test, y_test, scalers['target'], seq_length
# Assuming 'data' is a DataFrame containing the necessary data
if 'Symbol' in data.columns:
    symbols = data['Symbol'].unique()
else:
    symbols = []

for symbol in symbols:
    print(f"Processing {symbol}")
    for case in cases:
        print(f"Case {case}")
        self_attention_lstm_model, self_attention_lstm_history, self_attention_bi_lstm_model, self_attention_bi_lstm_history, X_test, y_test, scaler_target, seq_length = process_and_train_model(data, symbol, case=case)

        # Evaluate LSTM model
        lstm_mse, lstm_rmse, lstm_mae, lstm_r2 = evaluate_model(self_attention_lstm_model, X_test, y_test, scaler_target, symbol, case, 'Self-Attention LSTM')
        # Evaluate Bi-LSTM model
        bi_lstm_mse, bi_lstm_rmse, bi_lstm_mae, bi_lstm_r2 = evaluate_model(self_attention_bi_lstm_model, X_test, y_test, scaler_target, symbol, case, 'Self-Attention Bi-LSTM')

        print(f"In case {case}: {symbol} Self-Attention LSTM have MSE={lstm_mse}, MAE={lstm_mae}, RMSE={lstm_rmse}")
        print(f"In case {case}: {symbol} Self-Attention BiLSTM have MSE={bi_lstm_mse}, MAE={bi_lstm_mae}, RMSE={bi_lstm_rmse}")

        # Store model information
        model_info_list[case].append({
            'model_type': 'Self-Attention LSTM',
            'symbol': symbol,
            'mse': lstm_mse,
            'rmse': lstm_rmse,
            'mae': lstm_mae,
            # 'r2': lstm_r2
        })
        model_info_list[case].append({
            'model_type': 'Self-Attention Bi-LSTM',
            'symbol': symbol,
            'mse': bi_lstm_mse,
            'rmse': bi_lstm_rmse,
            'mae': bi_lstm_mae,
            # 'r2': bi_lstm_r2
        })

# Print all model information for each case
model_info_list = {}  # Define the model_info_list variable as an empty dictionary before using it

for case, models in model_info_list.items():
    print(f"\nIn case {case}:")
    for model_info in models:
        print(f"{model_info['model_type']} model for symbol {model_info['symbol']} - MSE: {model_info['mse']}, RMSE: {model_info['rmse']}, MAE: {model_info['mae']}")

Processing E1VFVN30
Case 1
Epoch 1/2
3/3 [==============================] - 11s 524ms/step - loss: 2.5855 - val_loss: 0.1477
Epoch 2/2
3/3 [==============================] - 0s 23ms/step - loss: 2.2225 - val_loss: 0.1309


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/2
3/3 [==============================] - 12s 1s/step - loss: 3.2276 - val_loss: 0.1303
Epoch 2/2
3/3 [==============================] - 0s 34ms/step - loss: 2.0393 - val_loss: 0.1138
